### Prerequisites for running notebook

In [2]:
!pip install kora -q

     |████████████████████████████████| 57 kB 5.4 MB/s 
     |████████████████████████████████| 60 kB 7.5 MB/s 


In [1]:
##Standard imports for project
import pandas as pd
import numpy as np
from datetime import datetime

#Functions for use
from sklearn.base import clone

#models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

#Evaluation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, accuracy_score

#Feature selection
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

#Hyperparameter optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## Required execution for notebook

The below verifies if the notebook is being executed in a local environment (Anaconda) or if the notebook is being hosted (Google Drive), and sets certain variables based on the requirement ("cwd" being the reference of the project directory; the notebook is always assumed to be executed at the root or the highest level of the project folder)

In [2]:
#Red pill or blue pill

import os

is_drive = False
cwd = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/Datasets/"

while True:
  offon = input("Is this being run offline? (Y = offline (i.e. Jupyter notebook), N = online (i.e. Google Colab notebook)): ")
  try:
    if offon.lower() not in ["y", "n"]:
      raise ValueError
    else:
      if offon.lower() == "n":
        from google.colab import drive
        from kora import drive as drives
        drive.mount('/content/drive')
        is_drive = True
        cwd = str(drives.chdir_notebook())
        cwd = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/Datasets/"
      break
  except ValueError:
    print("Error! Please only type one of the following: Y, y, N, n")

Is this being run offline? (Y = offline (i.e. Jupyter notebook), N = online (i.e. Google Colab notebook)): Y


In [2]:
import os
cwd = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/Datasets/"

In [61]:
df1 = pd.read_csv(cwd+"milk dataset 1.csv")
df2 = pd.read_csv(cwd+"milk dataset 2.csv")
dfn = pd.read_csv(cwd+"netherlands dataset output.csv")

df1

In [62]:
df1

,Unnamed: 0,Year,Month,Average price of raw milk from Ireland (Euro per 100kg),Butter (Thousand tonnes),Calf nuts and cubes (16-18% protein) (Euro per Tonne),Cheese (Thousand tonnes),Cow slaughterings (Thousand tonnes),Dairy meal (16-18% protein) (Euro per Tonne),Dairy nuts and cubes (16-18% protein) (Euro per Tonne),...,Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake (Million litres),Maize meal (Euro per Tonne),Skimmed & semi-skimmed milk sales (Million litres),Skimmed milk powder (Thousand tonnes),Whole milk sales (Million litres)
0,0,2014,1,42.34,4.4,329.0,2.3,49.4,300.0,302.0,...,498.38,597.40,704.50,789.13,867.04,38.8,252.0,16.9,0.0,23.5
1,1,2014,2,41.76,6.2,329.0,6.5,45.4,303.0,303.0,...,506.55,612.72,704.99,790.58,884.23,37.6,248.0,15.4,0.8,21.3
2,2,2014,3,39.04,14.4,305.0,18.4,48.6,290.0,288.0,...,497.04,588.64,682.82,789.94,906.31,40.8,244.0,18.7,3.9,25.4
3,3,2014,4,38.55,17.3,313.0,22.8,49.3,276.0,289.0,...,472.67,589.84,689.85,806.49,908.97,43.6,233.0,15.0,6.5,21.9
4,4,2014,5,37.10,21.7,315.0,24.8,48.8,284.0,292.0,...,509.21,595.29,691.55,808.22,893.82,51.1,240.0,17.4,11.9,24.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,91,2021,8,39.23,28.8,355.0,29.3,53.3,316.0,324.0,...,480.00,598.07,689.20,801.61,913.59,0.0,293.0,15.8,16.4,29.0
92,92,2021,9,42.44,26.5,360.0,33.2,55.7,323.0,332.0,...,450.74,570.54,694.88,804.71,915.68,0.0,299.0,15.1,11.1,26.6
93,93,2021,10,46.52,21.6,365.0,27.5,53.5,328.0,339.0,...,516.00,529.67,656.53,777.37,890.95,0.0,309.0,15.8,5.5,26.1
94,94,2021,11,48.65,17.8,370.0,20.9,55.9,333.0,344.0,...,419.66,547.42,641.39,762.83,888.34,0.0,314.0,15.4,0.0,25.5


In [63]:
df2

,Unnamed: 0,Year,Month,Average price of raw milk from Ireland (Euro per 100kg),Butter (Thousand tonnes),Cheese (Thousand tonnes),Domestic milk intake (Million litres),Fat content (Percent),Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake (Million litres),Skimmed & semi-skimmed milk sales (Million litres),Skimmed milk powder (Thousand tonnes),Whole milk sales (Million litres)
0,0,2007,1,28.30,3.8,1.3,123.0,3.87,363.58,431.76,520.38,574.80,661.13,54.6,12.3,2.4,32.1
1,1,2007,2,27.18,5.0,2.4,185.4,3.87,386.08,462.94,539.81,587.47,646.99,38.0,12.6,1.9,29.3
2,2,2007,3,25.64,10.1,10.2,386.5,3.80,409.33,474.93,539.83,603.81,670.93,35.8,12.8,5.0,32.4
3,3,2007,4,27.29,14.8,17.6,581.0,3.59,399.75,457.74,535.86,625.49,690.35,38.7,12.1,10.1,30.4
4,4,2007,5,29.76,19.1,19.3,688.3,3.62,398.11,461.66,536.39,607.95,673.32,46.9,12.8,13.2,31.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,175,2021,8,39.23,28.8,29.3,917.4,4.19,480.00,598.07,689.20,801.61,913.59,0.0,15.8,16.4,29.0
176,176,2021,9,42.44,26.5,33.2,776.7,4.43,450.74,570.54,694.88,804.71,915.68,0.0,15.1,11.1,26.6
177,177,2021,10,46.52,21.6,27.5,652.8,4.77,516.00,529.67,656.53,777.37,890.95,0.0,15.8,5.5,26.1
178,178,2021,11,48.65,17.8,20.9,460.6,4.90,419.66,547.42,641.39,762.83,888.34,0.0,15.4,0.0,25.5


In [64]:
dfn

,Unnamed: 0,Year,Month,Volume (Thousand tonnes),Fat content (Percent),Butter (Thousand tonnes),Cheese (Thousand tonnes),Skimmed-milk powder (Thousand tonnes),Concentrated milk (Thousand tonnes)
0,0,2011,1,996.942,4.56,12.130,65.490,6.402,28.106
1,1,2011,2,913.932,4.52,10.194,58.800,4.254,29.770
2,2,2011,3,1006.645,4.52,10.427,66.373,4.245,30.161
3,3,2011,4,1009.174,4.39,10.274,63.096,5.559,31.287
4,4,2011,5,1029.062,4.28,11.098,65.061,6.264,32.218
...,...,...,...,...,...,...,...,...,...
127,127,2021,8,1123.391,4.29,10.956,76.877,5.583,32.654
128,128,2021,9,1064.517,4.34,9.105,74.810,6.392,31.881
129,129,2021,10,1087.524,4.47,11.243,77.558,5.545,31.831
130,130,2021,11,1059.600,4.53,9.579,75.359,8.460,31.664


In [65]:
df1 = df1[df1.columns[1:]]

In [66]:

df2 = df2[df2.columns[1:]]
dfn = dfn[dfn.columns[1:]]

In [67]:
df1

,Year,Month,Average price of raw milk from Ireland (Euro per 100kg),Butter (Thousand tonnes),Calf nuts and cubes (16-18% protein) (Euro per Tonne),Cheese (Thousand tonnes),Cow slaughterings (Thousand tonnes),Dairy meal (16-18% protein) (Euro per Tonne),Dairy nuts and cubes (16-18% protein) (Euro per Tonne),Domestic milk intake (Million litres),...,Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake (Million litres),Maize meal (Euro per Tonne),Skimmed & semi-skimmed milk sales (Million litres),Skimmed milk powder (Thousand tonnes),Whole milk sales (Million litres)
0,2014,1,42.34,4.4,329.0,2.3,49.4,300.0,302.0,132.0,...,498.38,597.40,704.50,789.13,867.04,38.8,252.0,16.9,0.0,23.5
1,2014,2,41.76,6.2,329.0,6.5,45.4,303.0,303.0,214.0,...,506.55,612.72,704.99,790.58,884.23,37.6,248.0,15.4,0.8,21.3
2,2014,3,39.04,14.4,305.0,18.4,48.6,290.0,288.0,470.7,...,497.04,588.64,682.82,789.94,906.31,40.8,244.0,18.7,3.9,25.4
3,2014,4,38.55,17.3,313.0,22.8,49.3,276.0,289.0,697.0,...,472.67,589.84,689.85,806.49,908.97,43.6,233.0,15.0,6.5,21.9
4,2014,5,37.10,21.7,315.0,24.8,48.8,284.0,292.0,785.5,...,509.21,595.29,691.55,808.22,893.82,51.1,240.0,17.4,11.9,24.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2021,8,39.23,28.8,355.0,29.3,53.3,316.0,324.0,917.4,...,480.00,598.07,689.20,801.61,913.59,0.0,293.0,15.8,16.4,29.0
92,2021,9,42.44,26.5,360.0,33.2,55.7,323.0,332.0,776.7,...,450.74,570.54,694.88,804.71,915.68,0.0,299.0,15.1,11.1,26.6
93,2021,10,46.52,21.6,365.0,27.5,53.5,328.0,339.0,652.8,...,516.00,529.67,656.53,777.37,890.95,0.0,309.0,15.8,5.5,26.1
94,2021,11,48.65,17.8,370.0,20.9,55.9,333.0,344.0,460.6,...,419.66,547.42,641.39,762.83,888.34,0.0,314.0,15.4,0.0,25.5


In [68]:
df1.columns

Index(['Year', 'Month',
       'Average price of raw milk from Ireland (Euro per 100kg)',
       'Butter (Thousand tonnes)',
       'Calf nuts and cubes (16-18% protein) (Euro per Tonne)',
       'Cheese (Thousand tonnes)', 'Cow slaughterings (Thousand tonnes)',
       'Dairy meal (16-18% protein) (Euro per Tonne)',
       'Dairy nuts and cubes (16-18% protein) (Euro per Tonne)',
       'Domestic milk intake (Million litres)', 'Fat content (Percent)',
       'Heifers 200-249kg', 'Heifers 250-299kg', 'Heifers 300-349kg',
       'Heifers 350-399kg', 'Heifers 400-449kg',
       'Imported milk intake (Million litres)', 'Maize meal (Euro per Tonne)',
       'Skimmed & semi-skimmed milk sales (Million litres)',
       'Skimmed milk powder (Thousand tonnes)',
       'Whole milk sales (Million litres)'],
      dtype='object')

In [69]:
translations = {"Average price of raw milk from Ireland (Euro per 100kg)": "Raw milk price",
               "Butter (Thousand tonnes)": "Butter",
                "Cheese (Thousand tonnes)": "Cheese",
               "Calf nuts and cubes (16-18% protein) (Euro per Tonne)": "Calf nuts value",
               "Dairy meal (16-18% protein) (Euro per Tonne)": "Dairy meal value",
               "Dairy nuts and cubes (16-18% protein) (Euro per Tonne)": "Dairy nuts value",
               "Domestic milk intake (Million litres)": "Domestic milk intake",
               "Imported milk intake (Million litres)": "Imported milk intake",
               "Maize meal (Euro per Tonne)": "Maize meal value",
               "Skimmed & semi-skimmed milk sales (Million litres)": "Skimmed milk sales",
               "Skimmed milk powder (Thousand tonnes)": "Skimmed milk powder",
               "Whole milk sales (Million litres)": "Whole milk sales",
               "Skimmed-milk powder (Thousand tonnes)": "Skimmed milk powder",
               "Volume (Thousand tonnes)": "Milk production volume",
               "Concentrated milk (Thousand tonnes)": "Whole milk sales"}

In [70]:
df1 = df1.rename(translations, axis=1)

In [71]:
df1

,Year,Month,Raw milk price,Butter,Calf nuts value,Cheese,Cow slaughterings (Thousand tonnes),Dairy meal value,Dairy nuts value,Domestic milk intake,...,Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake,Maize meal value,Skimmed milk sales,Skimmed milk powder,Whole milk sales
0,2014,1,42.34,4.4,329.0,2.3,49.4,300.0,302.0,132.0,...,498.38,597.40,704.50,789.13,867.04,38.8,252.0,16.9,0.0,23.5
1,2014,2,41.76,6.2,329.0,6.5,45.4,303.0,303.0,214.0,...,506.55,612.72,704.99,790.58,884.23,37.6,248.0,15.4,0.8,21.3
2,2014,3,39.04,14.4,305.0,18.4,48.6,290.0,288.0,470.7,...,497.04,588.64,682.82,789.94,906.31,40.8,244.0,18.7,3.9,25.4
3,2014,4,38.55,17.3,313.0,22.8,49.3,276.0,289.0,697.0,...,472.67,589.84,689.85,806.49,908.97,43.6,233.0,15.0,6.5,21.9
4,2014,5,37.10,21.7,315.0,24.8,48.8,284.0,292.0,785.5,...,509.21,595.29,691.55,808.22,893.82,51.1,240.0,17.4,11.9,24.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2021,8,39.23,28.8,355.0,29.3,53.3,316.0,324.0,917.4,...,480.00,598.07,689.20,801.61,913.59,0.0,293.0,15.8,16.4,29.0
92,2021,9,42.44,26.5,360.0,33.2,55.7,323.0,332.0,776.7,...,450.74,570.54,694.88,804.71,915.68,0.0,299.0,15.1,11.1,26.6
93,2021,10,46.52,21.6,365.0,27.5,53.5,328.0,339.0,652.8,...,516.00,529.67,656.53,777.37,890.95,0.0,309.0,15.8,5.5,26.1
94,2021,11,48.65,17.8,370.0,20.9,55.9,333.0,344.0,460.6,...,419.66,547.42,641.39,762.83,888.34,0.0,314.0,15.4,0.0,25.5


In [72]:
df2 = df2.rename(translations, axis=1)
dfn = dfn.rename(translations, axis=1)

In [73]:
df2

,Year,Month,Raw milk price,Butter,Cheese,Domestic milk intake,Fat content (Percent),Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake,Skimmed milk sales,Skimmed milk powder,Whole milk sales
0,2007,1,28.30,3.8,1.3,123.0,3.87,363.58,431.76,520.38,574.80,661.13,54.6,12.3,2.4,32.1
1,2007,2,27.18,5.0,2.4,185.4,3.87,386.08,462.94,539.81,587.47,646.99,38.0,12.6,1.9,29.3
2,2007,3,25.64,10.1,10.2,386.5,3.80,409.33,474.93,539.83,603.81,670.93,35.8,12.8,5.0,32.4
3,2007,4,27.29,14.8,17.6,581.0,3.59,399.75,457.74,535.86,625.49,690.35,38.7,12.1,10.1,30.4
4,2007,5,29.76,19.1,19.3,688.3,3.62,398.11,461.66,536.39,607.95,673.32,46.9,12.8,13.2,31.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2021,8,39.23,28.8,29.3,917.4,4.19,480.00,598.07,689.20,801.61,913.59,0.0,15.8,16.4,29.0
176,2021,9,42.44,26.5,33.2,776.7,4.43,450.74,570.54,694.88,804.71,915.68,0.0,15.1,11.1,26.6
177,2021,10,46.52,21.6,27.5,652.8,4.77,516.00,529.67,656.53,777.37,890.95,0.0,15.8,5.5,26.1
178,2021,11,48.65,17.8,20.9,460.6,4.90,419.66,547.42,641.39,762.83,888.34,0.0,15.4,0.0,25.5


In [74]:
dfn

,Year,Month,Milk production volume,Fat content (Percent),Butter,Cheese,Skimmed milk powder,Whole milk sales
0,2011,1,996.942,4.56,12.130,65.490,6.402,28.106
1,2011,2,913.932,4.52,10.194,58.800,4.254,29.770
2,2011,3,1006.645,4.52,10.427,66.373,4.245,30.161
3,2011,4,1009.174,4.39,10.274,63.096,5.559,31.287
4,2011,5,1029.062,4.28,11.098,65.061,6.264,32.218
...,...,...,...,...,...,...,...,...
127,2021,8,1123.391,4.29,10.956,76.877,5.583,32.654
128,2021,9,1064.517,4.34,9.105,74.810,6.392,31.881
129,2021,10,1087.524,4.47,11.243,77.558,5.545,31.831
130,2021,11,1059.600,4.53,9.579,75.359,8.460,31.664


# 

In [75]:
df1["Milk production volume"] = [x+y for (x,y) in zip(df1["Domestic milk intake"],
                                                     df1["Imported milk intake"])]

In [76]:
df1["Milk production volume"]

0     170.8
1     251.6
2     511.5
3     740.6
4     836.6
      ...  
91    917.4
92    776.7
93    652.8
94    460.6
95    256.3
Name: Milk production volume, Length: 96, dtype: float64

In [77]:
df2["Milk production volume"] = [x+y for (x,y) in zip(df2["Domestic milk intake"],
                                                     df2["Imported milk intake"])]

In [78]:
dfn["Milk production volume"] = [x*.971164 for x in dfn["Milk production volume"]]
dfn["Whole milk sales"] = [x*.971164 for x in dfn["Whole milk sales"]]

In [79]:
dfn["Milk production volume"]

0       968.194180
1       887.577857
2       977.617385
3       980.073459
4       999.387968
          ...     
127    1090.996897
128    1033.820588
129    1056.164158
130    1029.045374
131    1089.158484
Name: Milk production volume, Length: 132, dtype: float64

### Target value

Cost of raw milk

# Tests

### Test 1 (on df1)

In [29]:
X = df1[(list(df1.columns[3:-1]))+["Month"]]
y = df1["Raw milk price"]

stratify = df1["Month"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22122,
                                                    stratify=stratify)

In [30]:
X

,Butter,Cheese,Domestic milk intake,Fat content (Percent),Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake,Skimmed milk sales,Skimmed milk powder,Month
0,3.8,1.3,123.0,3.87,363.58,431.76,520.38,574.80,661.13,54.6,12.3,2.4,1
1,5.0,2.4,185.4,3.87,386.08,462.94,539.81,587.47,646.99,38.0,12.6,1.9,2
2,10.1,10.2,386.5,3.80,409.33,474.93,539.83,603.81,670.93,35.8,12.8,5.0,3
3,14.8,17.6,581.0,3.59,399.75,457.74,535.86,625.49,690.35,38.7,12.1,10.1,4
4,19.1,19.3,688.3,3.62,398.11,461.66,536.39,607.95,673.32,46.9,12.8,13.2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,28.8,29.3,917.4,4.19,480.00,598.07,689.20,801.61,913.59,0.0,15.8,16.4,8
176,26.5,33.2,776.7,4.43,450.74,570.54,694.88,804.71,915.68,0.0,15.1,11.1,9
177,21.6,27.5,652.8,4.77,516.00,529.67,656.53,777.37,890.95,0.0,15.8,5.5,10
178,17.8,20.9,460.6,4.90,419.66,547.42,641.39,762.83,888.34,0.0,15.4,0.0,11


In [31]:
X_train

,Butter,Cheese,Domestic milk intake,Fat content (Percent),Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake,Skimmed milk sales,Skimmed milk powder,Month
116,18.5,21.9,583.8,4.33,435.00,566.38,669.24,750.59,855.62,56.7,17.2,7.7,9
38,10.1,15.5,382.1,3.89,374.87,452.19,515.64,583.73,657.74,33.9,15.4,1.3,3
155,10.9,5.4,241.4,4.58,449.38,493.91,601.98,720.53,831.40,0.0,16.1,10.2,12
160,29.8,31.6,1115.3,3.87,593.33,661.74,741.01,853.84,903.46,0.0,15.7,24.6,5
93,13.5,14.8,426.3,4.49,508.42,584.94,679.65,774.27,863.35,37.4,16.5,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,20.3,23.3,699.3,4.08,540.33,605.63,696.53,785.09,884.48,50.8,18.2,11.0,8
48,4.0,2.5,146.7,3.99,445.22,541.00,614.39,663.88,737.82,28.8,14.5,0.0,1
108,5.0,2.0,147.6,4.17,534.57,660.13,748.13,817.96,902.65,78.6,17.8,4.4,1
154,15.1,23.5,418.1,4.84,367.00,475.32,580.84,685.79,792.25,0.0,16.9,0.0,11


In [32]:
y_test

14     39.64
148    31.47
62     33.41
133    37.00
173    37.39
175    39.23
74     34.09
131    40.59
69     35.74
57     38.26
1      27.18
34     28.63
92     35.45
71     33.70
87     38.55
150    31.08
104    29.23
84     42.34
9      44.60
63     30.88
144    34.28
42     31.20
166    39.14
52     32.05
122    31.66
127    37.68
145    34.38
70     35.93
137    31.66
44     33.67
30     22.04
5      32.64
46     34.18
151    31.95
163    33.89
11     43.25
27     22.34
136    31.17
4      29.76
0      28.30
85     41.76
111    24.08
141    38.75
174    37.20
68     33.31
Name: Raw milk price, dtype: float64

#### Random Forest

##### 1. Basic test to test for suitability for remaining sections

In [33]:
#Random forest(basic features, randomized state, squared error scoring)
rf1 = RandomForestRegressor(criterion="absolute_error")

#Random forest, 500 estimators, 20 samples to split a node, min 5 samples at
#leaf nodes, squared error scoring, non-random state set
rf2 = RandomForestRegressor(n_estimators=500, min_samples_split=20, 
                             min_samples_leaf=5, n_jobs=-1, 
                             criterion="absolute_error", random_state=22122)

#Random forest, 1000 estimators, squared error scoring, non-random state set
rf3 = RandomForestRegressor(n_estimators=1000, n_jobs=-1,
                            criterion="absolute_error", random_state=22122)

##### Cross-validation tests (accuracy and mean average error)

In [34]:
(np.mean(cross_val_score(rf1, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf1, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.5s finished


(0.057592273234713366, -3.798604965465464)

In [35]:
(np.mean(cross_val_score(rf2, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf2, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   2.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.3s finished


(0.08870893225157864, -3.7830393492158843)

In [36]:
(np.mean(cross_val_score(rf3, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf3, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] END .................................................... total time=   0.7s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.5s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.7s
[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.4s finished


(0.05973401623878072, -3.770529270704027)

In [37]:
rf1.fit(X_train, y_train)
y1 = rf1.predict(X_test)

In [38]:
y1

array([33.01705   , 34.0124    , 31.4754    , 35.0044    , 32.2909    ,
       34.6654    , 32.756     , 40.3153    , 37.4767    , 36.4614    ,
       28.8489    , 33.7214    , 36.01715   , 34.95254381, 34.7031    ,
       32.6534    , 32.7183    , 36.1678    , 28.60924072, 29.997     ,
       35.12585   , 30.30137835, 40.5613    , 34.53705   , 28.6844    ,
       32.1849    , 35.6726    , 37.5791    , 33.6693    , 32.18043144,
       27.33034691, 31.02499381, 36.72584691, 33.6625    , 33.3747    ,
       32.79152835, 29.10494691, 32.6184    , 30.21819691, 28.73199381,
       34.3339    , 31.2145    , 37.2941    , 33.9291    , 36.5775    ])

In [39]:
mean_absolute_error(y_test, y1)

3.173284501835168

In [40]:
rf2.fit(X_train, y_train)
y2 = rf2.predict(X_test)

In [41]:
y2

array([33.86239567, 33.12164567, 29.86570345, 34.43993691, 32.24323814,
       34.30812691, 33.08194629, 37.92143753, 36.84634098, 36.35455629,
       29.28451629, 31.50138134, 34.9109685 , 35.08808479, 34.08332691,
       32.84836912, 33.83786098, 34.55160505, 28.79076789, 29.69604407,
       34.21855505, 29.93769603, 38.28481876, 34.04598629, 30.80700691,
       32.84082629, 35.58234753, 36.74815381, 33.6232116 , 31.27822763,
       27.73915691, 29.82092912, 34.81382789, 33.6535985 , 33.29745691,
       28.60179356, 30.32869974, 33.42047691, 29.54797443, 28.82898629,
       34.35767505, 32.55611629, 36.81162283, 33.08151753, 34.92139727])

In [42]:
mean_absolute_error(y_test, y2)

3.3263112108108097

In [43]:
rf2.score(X_test, y_test)

0.12873742511534458

In [44]:
rf3.fit(X_train, y_train)
y3 = rf3.predict(X_test)

In [45]:
y3

array([33.88343345, 33.543515  , 31.46519   , 34.853565  , 32.729385  ,
       34.685475  , 32.820965  , 39.52419469, 38.10111   , 36.6002    ,
       28.00110691, 33.82469469, 35.87848   , 35.5496032 , 34.397125  ,
       32.33861   , 32.9427    , 35.36072907, 28.10811407, 30.3211    ,
       35.143315  , 30.90412258, 41.5913    , 34.72124   , 29.60119   ,
       32.25571   , 36.17858   , 37.49678876, 33.609675  , 32.34987082,
       27.12305   , 30.26993283, 37.34172258, 33.64135   , 33.56427   ,
       31.5934118 , 28.72687938, 33.256505  , 29.92703876, 28.46239196,
       34.505735  , 31.12536   , 37.14515   , 33.676725  , 36.67651469])

In [46]:
mean_absolute_error(y_test, y3)

3.227020813346702

In [47]:
rf4 = clone(rf2)

In [48]:
#Create Random Feature elimination for above model
selector1 = RFECV(rf4, cv=5, step=1, n_jobs=-1, verbose=2, min_features_to_select=7)

#Fit all data
selector1.fit(X, y)

Fitting estimator with 13 features.


RFECV(cv=5,
      estimator=RandomForestRegressor(criterion='absolute_error',
                                      min_samples_leaf=5, min_samples_split=20,
                                      n_estimators=500, n_jobs=-1,
                                      random_state=22122),
      min_features_to_select=7, n_jobs=-1, verbose=2)

In [49]:
selector1.ranking_

array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [53]:
X.columns

Index(['Butter', 'Cheese', 'Domestic milk intake', 'Fat content (Percent)',
       'Heifers 200-249kg', 'Heifers 250-299kg', 'Heifers 300-349kg',
       'Heifers 350-399kg', 'Heifers 400-449kg', 'Imported milk intake',
       'Skimmed milk sales', 'Skimmed milk powder', 'Month'],
      dtype='object')

In [54]:
rankings = selector1.ranking_
columns_to_choose = [X.columns[i] for i in range(len(X.columns)) if rankings[i] == 1]
rfe_rfX = X[columns_to_choose]
rfe_rfX_train = X_train[columns_to_choose]
rfe_rfX_test = X_test[columns_to_choose]

In [55]:
columns_to_choose

['Butter',
 'Domestic milk intake',
 'Fat content (Percent)',
 'Heifers 200-249kg',
 'Heifers 250-299kg',
 'Heifers 300-349kg',
 'Heifers 350-399kg',
 'Heifers 400-449kg',
 'Imported milk intake',
 'Skimmed milk sales',
 'Skimmed milk powder',
 'Month']

In [56]:
(np.mean(cross_val_score(rf4, rfe_rfX, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf4, rfe_rfX, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.4s finished


(0.10021154320846926, -3.7615774554554577)

In [57]:
rf4.fit(rfe_rfX_train, y_train)
y4 = rf4.predict(rfe_rfX_test)

In [58]:
y4

array([33.91495567, 33.05122222, 29.67598469, 34.38548691, 32.26446345,
       34.21439283, 33.15117629, 37.92154876, 36.89364753, 36.38381222,
       29.10790098, 31.61915283, 35.04005912, 34.92230665, 34.37141629,
       32.82639036, 33.74165753, 34.61022098, 28.86819814, 29.55572469,
       34.28879098, 29.82072381, 38.35769   , 34.14108629, 30.79077345,
       32.97949876, 35.55032753, 36.78970098, 33.54397283, 31.09909072,
       27.86017938, 29.75596505, 35.03179629, 33.54272381, 33.45458345,
       28.45000036, 30.26836505, 33.31950222, 29.42319036, 28.67592753,
       34.45272567, 32.50275814, 37.03246469, 32.97893345, 34.98101912])

In [59]:
mean_absolute_error(y4, y_test)

3.3118966311644966

In [60]:
n_estimators = [i for i in range(100, 1001, 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [i for i in range(10, 101, 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 4, 5, 7, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Criterion to select scoring
criterion = ["absolute_error", "squared_error", "gini"]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

In [62]:
rf_best = clone(gs_random.best_estimator_)

In [63]:
gs_random.best_estimator_

RandomForestRegressor(criterion='absolute_error', max_depth=20,
                      max_features='sqrt', min_samples_split=10,
                      n_estimators=200, random_state=22122)

##### Cross validation (both accuracy and absolute error)

In [ ]:
(np.mean(cross_val_score(rf_best, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf_best, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

###### Fit

In [ ]:
rf_best.fit(X_train, y_train)
ybrf = rf4.predict(X_test)

###### Results

In [ ]:
ybrf

###### Absolute error

In [ ]:
mean_absolute_error(ybrf, y_test)

##### 4. Best model from 3 with RFE applied

##### Apply RFE (with cross-validation) 

In [ ]:
#Clone best performing model of previous section
rf_best2 = clone(rf_best)

#Create Random Feature elimination for above model
selector = RFECV(rf_best2, cv=5, step=1, n_jobs=-1, verbose=2, min_features_to_select=len(X.columns) // 2)

#Fit all data
selector.fit(X, y)

In [ ]:
selector.ranking_

In [ ]:
X.columns

##### Define model with new data

In [ ]:
rankings = selector.ranking_
columns_to_choose = [X.columns[i] for i in range(len(X.columns)) if rankings[i] == 1]
rfe_rfX_best = X[columns_to_choose]
rfe_rfX_best_train = X_train[columns_to_choose]
rfe_rfX_best_test = X_test[columns_to_choose]

##### Cross validation (both accuracy and absolute error)

In [ ]:
(np.mean(cross_val_score(rf_best2, rfe_rfX, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf_best2, rfe_rfX, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

###### Fit

In [ ]:
rf_best2.fit(rfe_rfX_best_train, y_train)
ybrf2 = rf4.predict(rfe_rfX_best_test)

###### Results

In [ ]:
ybrf2

###### Absolute error

In [ ]:
mean_absolute_error(ybrf2, y_test)

### K-Nearest Neighbour Regression

### 1. Basic test models with all features

In [ ]:
#KNeighbors Regressor, default parameters
knn1 = KNeighborsRegressor(n_jobs=-1)

#KNeighbors Regressor, estimates from 10 nearest neighbours, each leaf has 10
#samples
knn2 = KNeighborsRegressor(n_neighbors=10, leaf_size=50, n_jobs=-1)

#KNeighbors Regressor, estimates from 10 nearest neighbours
knn3 = KNeighborsRegressor(n_neighbors=20, n_jobs=-1)

#### Cross validation tests (both accuracy and average error)

In [ ]:
(np.mean(cross_val_score(knn1, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn1, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.7s finished


(0.9180921608506607, -368.379723482743)

In [ ]:
(np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished


(0.9247140939410677, -350.91984476224167)

In [ ]:
(np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished


(0.9247140939410677, -350.91984476224167)

#### KNeighbors Regressor 1

###### Fit

In [ ]:
knn1.fit(X_train, y_train)
yk1 = knn1.predict(X_test)

###### Results

In [ ]:
yk1

array([4944.2,  692.8, 2102.6, 3167.2, 3299.8, 5318. , 6853.8,  668.6,
       1491. , 4594.8, 7215. , 6197.6, 4340.8, 4506. , 5739.2, 1372.8,
       1601.6,  571. , 3981.2, 1243.4, 6257. , 4437.2,  686.2, 5202.8,
       2200. , 6881.8,  550.4, 5128.2, 5024.4, 4883.8, 6692. , 5513.4,
       2963.8, 6775.8, 5420.2, 6517.4, 4489. , 1334.8, 3688.4, 1923.4,
       4225.4, 5793.6, 2972.6, 1856.8, 5006.2, 3305.6,  933.4, 2379.2,
       6778.8, 6631.6, 5943. , 6861.8, 4980. , 4766.2, 6220.2, 5546.2,
       5636.2,  691. , 1980.6, 1650.2, 1595.2, 4388.4, 5740.6, 4904.6,
       6417.4, 5408.6, 3682.8,  601.4, 5088.4, 2728.8, 3186.2, 5343.8,
        493.2,  673.2, 3244. , 3715.4, 1555.8, 4296.6, 1588.2, 3071.8,
       6630.8, 3454.2, 1852.8, 6622.2, 1722. , 5907.2, 5768.8, 1785.6,
       2684.2, 6338.6, 4792. , 2179. , 2830.6, 4069.2, 5125.4, 6479.8,
       6954.4, 3680.8, 3090. , 4081.4, 2904.6, 2272.2, 1492. , 3108.8,
       6630.6, 6145.4, 6076.6, 6489. , 2804.6, 5221.6, 6508. , 4055.8,
      

###### Absolute error

In [ ]:
mean_absolute_error(yk1, y_test)

311.5599169262721

#### KNeighbors Regressor 2

###### Fit

In [ ]:
knn2.fit(X_train, y_train)
yk2 = knn2.predict(X_test)

###### Results

In [ ]:
yk2

array([4911.3,  690.3, 2109.4, 3204.6, 3353.4, 5688.3, 6481.2,  692.5,
       1533.9, 4616.5, 7034.2, 5783.6, 4442.9, 4831.4, 5801.3, 1341.8,
       1566.8,  576.4, 4167.7, 1325.5, 6277.8, 4377.6,  669.8, 5185.4,
       2063.1, 6811.3,  592.6, 5090.9, 5224.2, 5037.7, 6529.5, 5418.8,
       3002. , 6761.3, 5423.6, 6033.8, 4664.3, 1322.8, 3648.3, 1907. ,
       4281.4, 5391.1, 2823.7, 1798.8, 5044.9, 3241.6,  864.5, 2440.8,
       6385.3, 6340. , 6213.1, 6885.7, 5334.2, 4661.3, 5971.5, 5202. ,
       5381.2,  668.2, 1906.9, 1575.3, 1601.4, 4173.6, 5617.6, 4949.6,
       6412.9, 5451.4, 3615.6,  575.6, 5138.3, 2750.7, 3124. , 5390.1,
        436. ,  688.7, 3199.6, 3832. , 1567.3, 4376.6, 1530.8, 3396. ,
       6644.2, 3470.2, 1930.1, 6505.5, 1789.8, 5910.8, 5534.3, 1813.4,
       2694.5, 6161.2, 4858.6, 2307.6, 2791.4, 4184.7, 4945.7, 6347.7,
       6942.9, 3862.2, 2836.6, 3897.4, 2968.4, 2194.8, 1522.8, 3066.3,
       6534.1, 5708.3, 5353.6, 6218.7, 2856.6, 5272. , 5964.3, 4004.4,
      

###### Absolute error

In [ ]:
mean_absolute_error(yk2, y_test)

306.98068535825547

#### KNeighbors Regressor 3

###### Fit

In [ ]:
knn3.fit(X_train, y_train)
yk3 = knn3.predict(X_test)

###### Results

In [ ]:
yk3

array([5178.35,  671.2 , 2075.75, 3040.35, 3402.15, 5539.  , 6439.  ,
        695.  , 1582.65, 4600.4 , 6848.95, 5727.  , 4457.8 , 5088.  ,
       5802.15, 1346.95, 1609.3 ,  564.35, 3972.7 , 1377.5 , 6279.7 ,
       4416.5 ,  672.25, 4971.  , 2106.4 , 6635.3 ,  618.8 , 5276.05,
       5253.9 , 4903.1 , 6425.75, 5540.15, 3189.4 , 7008.85, 5453.65,
       6225.15, 4547.2 , 1299.75, 3713.2 , 1892.3 , 4417.6 , 5407.35,
       2832.45, 1828.4 , 5349.6 , 3239.25,  837.  , 2510.1 , 6577.15,
       6403.35, 6375.6 , 6727.4 , 5129.35, 4661.15, 5929.3 , 5308.25,
       5312.05,  650.15, 1946.55, 1590.2 , 1579.95, 4108.4 , 5538.15,
       5082.85, 6339.55, 5742.15, 3711.95,  581.65, 5494.15, 2754.1 ,
       3134.75, 5558.25,  426.95,  691.4 , 3416.95, 3764.4 , 1573.25,
       4191.2 , 1524.55, 4070.9 , 6717.9 , 3486.5 , 2012.3 , 6337.1 ,
       1777.05, 5811.55, 5448.25, 1946.1 , 2704.1 , 6256.5 , 4760.8 ,
       2287.4 , 2717.2 , 4048.3 , 4512.7 , 6378.55, 6944.5 , 3864.6 ,
       2861.1 , 3840

###### Absolute error

In [ ]:
mean_absolute_error(yk3, y_test)

301.3296469366563

### GridSearchCV

##### Parameter grid

In [ ]:
# Loss
weights = ["uniform", "distance"]
# Number of estimators
n_neighbours = [1, 2, 5, 7, 10, 14, 15, 20]
# Criterion for measuring quality of split
p = [1, 2]
# Minimum number of samples required to split a node
leaf_size = [i for i in range(10, 50, 10)]
# Create the random grid
knn_grid = {'weights': weights,
            'n_neighbors': n_neighbours,
            'p': p,
            'leaf_size': leaf_size}

##### GridSearchCV algorithm

In [ ]:
knn = KNeighborsRegressor(n_jobs=-1)

knn_gs = GridSearchCV(estimator = knn, param_grid = knn_grid,
                      scoring="neg_mean_absolute_error",
                      n_jobs=-1, cv=KFold(5), return_train_score=True,
                      verbose=4)

knn_gs.fit(X, y)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=KNeighborsRegressor(n_jobs=-1), n_jobs=-1,
             param_grid={'leaf_size': [10, 20, 30, 40],
                         'n_neighbors': [1, 2, 5, 7, 10, 14, 15, 20],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=4)

In [ ]:
knn_gs.best_estimator_.get_params()

{'algorithm': 'auto',
 'leaf_size': 10,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': -1,
 'n_neighbors': 20,
 'p': 1,
 'weights': 'distance'}

### 2. Best model

##### Defining best model

In [ ]:
knn_best = KNeighborsRegressor(algorithm='auto', leaf_size=10, 
                               metric='minkowski', metric_params=None, 
                               n_jobs=-1, p=1, weights='distance')

##### Cross validation (both accuracy and absolute error)

In [ ]:
(np.mean(cross_val_score(knn_best, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn_best, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.9s finished


(0.9238901692717343, -346.3115188195462)

###### Fit

In [ ]:
knn_best.fit(X_train, y_train)
ybknn = knn_best.predict(X_test)

###### Results

In [ ]:
ybknn

array([5376.44979017,  701.87257425, 2153.51084861, 3161.75779251,
       3334.66238808, 5343.21560057, 6250.23689914,  675.00320328,
       1496.26840723, 4304.83403693, 7140.27682663, 6200.1926661 ,
       4648.13800993, 4554.63431667, 5539.92262326, 1399.84225884,
       1617.86121026,  566.03552037, 4162.23938629, 1222.5059959 ,
       6269.71984667, 4490.68095016,  677.91166079, 5336.6500824 ,
       2445.38607356, 6905.20173927,  340.86180087, 5109.01010211,
       5327.22182454, 5154.64671932, 6346.03993582, 5515.24103973,
       2917.13351733, 6677.33379093, 5613.70154255, 6525.10301698,
       4291.19712643, 1249.52236649, 3666.96027792, 1849.03001107,
       4271.46528147, 5655.68225431, 2615.0679525 , 1847.7436953 ,
       5142.28434374, 3187.04836719,  947.55259071, 2242.50985428,
       6777.34732875, 6197.96978902, 6440.72152239, 6774.8255985 ,
       5023.9617116 , 4913.9856626 , 6130.83415221, 5559.73018772,
       5446.7599344 ,  694.9110202 , 1975.98957356, 1654.19028

###### Absolute error

In [ ]:
mean_absolute_error(ybknn, y_test)

290.824070365135

### Test 2 (on df2)

In [83]:
X = df2[(list(df2.columns[3:-1]))+["Month"]]
y = df2["Raw milk price"]

stratify = df2["Month"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=22122,
                                                    stratify=stratify)

In [84]:
X

,Butter,Cheese,Domestic milk intake,Fat content (Percent),Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake,Skimmed milk sales,Skimmed milk powder,Whole milk sales,Month
0,3.8,1.3,123.0,3.87,363.58,431.76,520.38,574.80,661.13,54.6,12.3,2.4,32.1,1
1,5.0,2.4,185.4,3.87,386.08,462.94,539.81,587.47,646.99,38.0,12.6,1.9,29.3,2
2,10.1,10.2,386.5,3.80,409.33,474.93,539.83,603.81,670.93,35.8,12.8,5.0,32.4,3
3,14.8,17.6,581.0,3.59,399.75,457.74,535.86,625.49,690.35,38.7,12.1,10.1,30.4,4
4,19.1,19.3,688.3,3.62,398.11,461.66,536.39,607.95,673.32,46.9,12.8,13.2,31.8,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,28.8,29.3,917.4,4.19,480.00,598.07,689.20,801.61,913.59,0.0,15.8,16.4,29.0,8
176,26.5,33.2,776.7,4.43,450.74,570.54,694.88,804.71,915.68,0.0,15.1,11.1,26.6,9
177,21.6,27.5,652.8,4.77,516.00,529.67,656.53,777.37,890.95,0.0,15.8,5.5,26.1,10
178,17.8,20.9,460.6,4.90,419.66,547.42,641.39,762.83,888.34,0.0,15.4,0.0,25.5,11


In [85]:
X_train

,Butter,Cheese,Domestic milk intake,Fat content (Percent),Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake,Skimmed milk sales,Skimmed milk powder,Whole milk sales,Month
116,18.5,21.9,583.8,4.33,435.00,566.38,669.24,750.59,855.62,56.7,17.2,7.7,27.2,9
38,10.1,15.5,382.1,3.89,374.87,452.19,515.64,583.73,657.74,33.9,15.4,1.3,29.1,3
155,10.9,5.4,241.4,4.58,449.38,493.91,601.98,720.53,831.40,0.0,16.1,10.2,25.8,12
160,29.8,31.6,1115.3,3.87,593.33,661.74,741.01,853.84,903.46,0.0,15.7,24.6,31.6,5
93,13.5,14.8,426.3,4.49,508.42,584.94,679.65,774.27,863.35,37.4,16.5,0.0,23.6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,20.3,23.3,699.3,4.08,540.33,605.63,696.53,785.09,884.48,50.8,18.2,11.0,26.8,8
48,4.0,2.5,146.7,3.99,445.22,541.00,614.39,663.88,737.82,28.8,14.5,0.0,27.6,1
108,5.0,2.0,147.6,4.17,534.57,660.13,748.13,817.96,902.65,78.6,17.8,4.4,24.7,1
154,15.1,23.5,418.1,4.84,367.00,475.32,580.84,685.79,792.25,0.0,16.9,0.0,26.8,11


In [86]:
y_test

14     39.64
148    31.47
62     33.41
133    37.00
173    37.39
175    39.23
74     34.09
131    40.59
69     35.74
57     38.26
1      27.18
34     28.63
92     35.45
71     33.70
87     38.55
150    31.08
104    29.23
84     42.34
9      44.60
63     30.88
144    34.28
42     31.20
166    39.14
52     32.05
122    31.66
127    37.68
145    34.38
70     35.93
137    31.66
44     33.67
30     22.04
5      32.64
46     34.18
151    31.95
163    33.89
11     43.25
27     22.34
136    31.17
4      29.76
0      28.30
85     41.76
111    24.08
141    38.75
174    37.20
68     33.31
Name: Raw milk price, dtype: float64

#### Random Forest

##### 1. Basic test to test for suitability for remaining sections

In [87]:
#Random forest(basic features, randomized state, squared error scoring)
rf1 = RandomForestRegressor(criterion="absolute_error")

#Random forest, 500 estimators, 20 samples to split a node, min 5 samples at
#leaf nodes, squared error scoring, non-random state set
rf2 = RandomForestRegressor(n_estimators=500, min_samples_split=20, 
                             min_samples_leaf=5, n_jobs=-1, 
                             criterion="absolute_error", random_state=22122)

#Random forest, 1000 estimators, squared error scoring, non-random state set
rf3 = RandomForestRegressor(n_estimators=1000, n_jobs=-1,
                            criterion="absolute_error", random_state=22122)

##### Cross-validation tests (accuracy and mean average error)

In [88]:
(np.mean(cross_val_score(rf1, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf1, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.3s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.3s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.6s finished


(0.025556105110798488, -3.842829240907574)

In [89]:
(np.mean(cross_val_score(rf2, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf2, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.4s finished


(0.07397912880826814, -3.8190363790123465)

In [90]:
(np.mean(cross_val_score(rf3, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf3, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.5s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV] END .................................................... total time=   0.8s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.7s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.5s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.7s
[CV] END .................................................... total time=   0.7s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.4s finished


(0.024434294168202086, -3.847246824841489)

In [91]:
rf1.fit(X_train, y_train)
y1 = rf1.predict(X_test)

In [92]:
y1

array([34.74599381, 33.0581    , 31.0757    , 35.4088    , 33.8904    ,
       34.1312    , 33.1917    , 39.7291    , 37.0644    , 35.8978    ,
       28.47344072, 32.39314691, 37.9502    , 36.00549381, 35.7566    ,
       32.4322    , 33.9661    , 35.9773    , 30.53323453, 30.1555    ,
       35.5545    , 30.08524691, 41.346     , 33.6169    , 29.0662    ,
       31.5355    , 36.2563    , 36.1435    , 32.6667    , 32.24629381,
       26.11234691, 30.8336    , 36.60059381, 33.1083    , 33.3637    ,
       32.11619742, 29.6289    , 32.7555    , 29.9034    , 30.91238144,
       35.2892    , 31.0299    , 36.9887    , 33.0775    , 37.4262    ])

In [93]:
mean_absolute_error(y_test, y1)

3.10975638038038

In [94]:
rf2.fit(X_train, y_train)
y2 = rf2.predict(X_test)

In [95]:
y2

array([33.8184685 , 32.93777876, 29.84692469, 34.23161753, 32.03387938,
       33.87410753, 32.92572222, 37.76806938, 36.85645814, 36.26147283,
       29.34980912, 31.5043116 , 35.6217016 , 35.09462665, 34.96523222,
       32.78119567, 34.09342407, 34.86300283, 29.05714098, 29.79001   ,
       33.87035567, 29.80120789, 38.13691   , 33.48722691, 30.838     ,
       32.65791814, 35.48321283, 36.64178691, 33.37959876, 31.48534258,
       27.87919283, 29.94587814, 34.85864222, 33.45558629, 33.24407222,
       28.86157443, 30.59488691, 33.07645407, 29.76090753, 29.24660443,
       34.7523916 , 32.50456407, 36.81332407, 32.82997222, 35.5149316 ])

In [96]:
mean_absolute_error(y_test, y2)

3.309340131731728

In [97]:
rf3.fit(X_train, y_train)
y3 = rf3.predict(X_test)

In [98]:
y3

array([34.27621783, 33.299625  , 31.217175  , 34.81404   , 32.925365  ,
       33.74355   , 32.669975  , 39.38235   , 37.363165  , 36.45116   ,
       28.14778567, 33.48032691, 37.53891   , 35.03198567, 35.85386   ,
       32.20647969, 33.25003   , 35.50182   , 30.37642381, 30.493355  ,
       34.73925   , 30.55473196, 41.674865  , 33.711775  , 29.35486   ,
       32.04023   , 35.9455    , 36.89729469, 32.904945  , 32.06290356,
       26.29558345, 30.50896814, 36.78558505, 33.055425  , 33.0138    ,
       31.60775324, 29.19418938, 32.75617   , 30.64047938, 29.68062541,
       35.22256   , 31.15919   , 37.650095  , 33.454865  , 37.24251938])

In [99]:
mean_absolute_error(y_test, y3)

3.1212365808475306

In [100]:
rf4 = clone(rf3)

In [101]:
#Create Random Feature elimination for above model
selector1 = RFECV(rf4, cv=5, step=1, n_jobs=-1, verbose=2, min_features_to_select=7)

#Fit all data
selector1.fit(X, y)

Fitting estimator with 14 features.


RFECV(cv=5,
      estimator=RandomForestRegressor(criterion='absolute_error',
                                      n_estimators=1000, n_jobs=-1,
                                      random_state=22122),
      min_features_to_select=7, n_jobs=-1, verbose=2)

In [102]:
selector1.ranking_

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [111]:
X.columns

Index(['Butter', 'Cheese', 'Domestic milk intake', 'Fat content (Percent)',
       'Heifers 200-249kg', 'Heifers 250-299kg', 'Heifers 300-349kg',
       'Heifers 350-399kg', 'Heifers 400-449kg', 'Imported milk intake',
       'Skimmed milk sales', 'Skimmed milk powder', 'Whole milk sales',
       'Month'],
      dtype='object')

In [112]:
rankings = selector1.ranking_
columns_to_choose = [X.columns[i] for i in range(len(X.columns)) if rankings[i] == 1]
rfe_rfX = X[columns_to_choose]
rfe_rfX_train = X_train[columns_to_choose]
rfe_rfX_test = X_test[columns_to_choose]

In [113]:
columns_to_choose

['Butter',
 'Cheese',
 'Fat content (Percent)',
 'Heifers 200-249kg',
 'Heifers 250-299kg',
 'Heifers 300-349kg',
 'Heifers 350-399kg',
 'Heifers 400-449kg',
 'Imported milk intake',
 'Skimmed milk sales',
 'Skimmed milk powder',
 'Whole milk sales',
 'Month']

In [106]:
(np.mean(cross_val_score(rf4, rfe_rfX, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf4, rfe_rfX, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.5s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.5s finished


(0.029547300346289208, -3.8378496902569097)

In [107]:
rf4.fit(rfe_rfX_train, y_train)
y4 = rf4.predict(rfe_rfX_test)

In [108]:
y4

array([34.28069098, 32.95301   , 31.17581   , 34.88100969, 32.82501   ,
       33.83783   , 32.45299   , 39.614305  , 37.36330469, 36.294185  ,
       28.12119629, 33.21153814, 37.48103   , 35.0328882 , 35.87242   ,
       32.29058469, 33.163975  , 35.48971469, 30.27340727, 30.39229   ,
       34.65517969, 30.72022134, 41.61203   , 33.70402   , 29.32339   ,
       32.02822   , 35.84177   , 36.76382907, 32.96359   , 32.02810417,
       26.35172469, 30.63005814, 36.83658443, 33.10071   , 33.08944   ,
       31.64358067, 29.03724   , 32.70638   , 30.85744407, 29.82773727,
       35.18587469, 31.13211   , 37.6098    , 33.87883   , 37.01706438])

In [109]:
mean_absolute_error(y4, y_test)

3.0917279771771886

In [117]:
n_estimators = [i for i in range(100, 1001, 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [i for i in range(10, 101, 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 4, 5, 7, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

In [118]:
rf = RandomForestRegressor(random_state=22122,
                           n_jobs=-1)

gs_random = GridSearchCV(estimator = rf, param_grid = random_grid,
                               scoring="neg_mean_absolute_error",
                               n_jobs=-1, cv=KFold(5), return_train_score=True,
                               verbose=4)

gs_random.fit(X, y)

Fitting 5 folds for each of 15840 candidates, totalling 79200 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestRegressor(n_jobs=-1, random_state=22122),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['absolute_error', 'squared_error'],
                         'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100,
                                       None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 3, 4, 5, 7, 10],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700,
                                          800, 900, 1000]},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=4)

In [120]:
rfr2 = RandomForestRegressor(random_state=22122,
                           n_jobs=-1)

gs_randomr2 = GridSearchCV(estimator = rf, param_grid = random_grid,
                               scoring="r2",
                               n_jobs=8, cv=KFold(5), return_train_score=True,
                               verbose=4)

gs_randomr2.fit(X, y)

Fitting 5 folds for each of 15840 candidates, totalling 79200 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestRegressor(n_jobs=-1, random_state=22122),
             n_jobs=8,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['absolute_error', 'squared_error'],
                         'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100,
                                       None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 3, 4, 5, 7, 10],
                         'n_estimators': [100, 200, 300, 400, 500, 600, 700,
                                          800, 900, 1000]},
             return_train_score=True, scoring='r2', verbose=4)

In [122]:
gs_randomr2.best_estimator_

RandomForestRegressor(max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=10, n_estimators=400, n_jobs=-1,
                      random_state=22122)

In [119]:
gs_random.best_estimator_

RandomForestRegressor(max_depth=10, max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=10, n_estimators=900, n_jobs=-1,
                      random_state=22122)

In [123]:
rf_best = clone(gs_random.best_estimator_)

##### Cross validation (both accuracy and absolute error)

In [124]:
(np.mean(cross_val_score(rf_best, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf_best, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   4.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.0s remaining:    0.0s


[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   0.8s
[CV] END .................................................... total time=   0.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    7.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.8s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   0.8s
[CV] END .................................................... total time=   0.8s
[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.4s finished


(0.1042549832330831, -3.6961069489397524)

In [125]:
X_train

,Butter,Cheese,Domestic milk intake,Fat content (Percent),Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake,Skimmed milk sales,Skimmed milk powder,Whole milk sales,Month
116,18.5,21.9,583.8,4.33,435.00,566.38,669.24,750.59,855.62,56.7,17.2,7.7,27.2,9
38,10.1,15.5,382.1,3.89,374.87,452.19,515.64,583.73,657.74,33.9,15.4,1.3,29.1,3
155,10.9,5.4,241.4,4.58,449.38,493.91,601.98,720.53,831.40,0.0,16.1,10.2,25.8,12
160,29.8,31.6,1115.3,3.87,593.33,661.74,741.01,853.84,903.46,0.0,15.7,24.6,31.6,5
93,13.5,14.8,426.3,4.49,508.42,584.94,679.65,774.27,863.35,37.4,16.5,0.0,23.6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,20.3,23.3,699.3,4.08,540.33,605.63,696.53,785.09,884.48,50.8,18.2,11.0,26.8,8
48,4.0,2.5,146.7,3.99,445.22,541.00,614.39,663.88,737.82,28.8,14.5,0.0,27.6,1
108,5.0,2.0,147.6,4.17,534.57,660.13,748.13,817.96,902.65,78.6,17.8,4.4,24.7,1
154,15.1,23.5,418.1,4.84,367.00,475.32,580.84,685.79,792.25,0.0,16.9,0.0,26.8,11


In [126]:
X_test

,Butter,Cheese,Domestic milk intake,Fat content (Percent),Heifers 200-249kg,Heifers 250-299kg,Heifers 300-349kg,Heifers 350-399kg,Heifers 400-449kg,Imported milk intake,Skimmed milk sales,Skimmed milk powder,Whole milk sales,Month
14,9.4,13.7,379.8,3.83,458.72,521.06,602.46,662.80,737.54,40.7,13.4,2.3,29.8,3
148,31.0,31.7,1072.2,3.85,446.33,562.41,670.29,782.83,910.06,55.5,17.2,24.2,28.2,5
62,13.9,18.1,473.0,3.99,643.39,753.95,836.96,926.91,1019.91,32.3,16.4,2.5,26.6,3
133,9.5,8.6,294.2,4.25,523.60,610.55,717.41,822.83,908.00,67.5,16.8,0.0,26.6,2
173,31.5,35.7,1067.3,3.97,485.51,608.60,730.25,844.54,965.24,0.0,15.0,19.3,29.7,6
175,28.8,29.3,917.4,4.19,480.00,598.07,689.20,801.61,913.59,0.0,15.8,16.4,29.0,8
74,12.1,17.9,468.1,3.97,522.05,619.77,732.02,826.26,942.24,37.1,15.9,3.0,25.3,3
131,11.4,5.0,205.5,4.57,467.38,551.46,636.88,765.40,877.70,0.0,17.3,0.0,27.1,12
69,10.0,15.0,364.3,4.34,446.00,516.86,631.03,730.85,851.75,28.3,15.6,0.0,25.3,10
57,11.4,12.2,377.7,4.33,524.12,627.50,679.09,762.47,831.43,26.7,15.2,3.1,26.3,10


###### Fit

In [127]:
rf_best.fit(X_train, y_train)
ybrf = rf_best.predict(X_test)

###### Results

In [128]:
ybrf

array([34.97108996, 33.19757135, 31.10815026, 34.5091964 , 32.3000421 ,
       34.35457088, 33.13079903, 38.79322207, 38.01543463, 37.40663725,
       28.60968499, 32.83186666, 37.0091843 , 36.25074888, 35.41937209,
       32.73911411, 33.74564663, 35.38891571, 31.14642433, 30.21448205,
       34.8226295 , 29.90862064, 39.9186443 , 34.01996226, 30.03947154,
       32.68681519, 35.37002482, 37.5837728 , 33.644747  , 32.73962207,
       28.58474592, 30.10816886, 35.11779974, 33.47001973, 33.31657645,
       31.46412047, 29.29349809, 33.47282074, 29.57890057, 29.57925069,
       35.1714623 , 31.63659309, 36.2372487 , 33.13248663, 36.9509887 ])

###### Absolute error

In [129]:
mean_absolute_error(ybrf, y_test)

3.1752967417493214

##### 4. Best model from 3 with RFE applied

##### Apply RFE (with cross-validation) 

In [130]:
#Clone best performing model of previous section
rf_best2 = clone(rf_best)

#Create Random Feature elimination for above model
selector = RFECV(rf_best2, cv=5, step=1, n_jobs=-1, verbose=2, min_features_to_select=len(X.columns) // 2)

#Fit all data
selector.fit(X, y)

Fitting estimator with 14 features.
Fitting estimator with 13 features.


RFECV(cv=5,
      estimator=RandomForestRegressor(max_depth=10, max_features='sqrt',
                                      min_samples_leaf=4, min_samples_split=10,
                                      n_estimators=900, n_jobs=-1,
                                      random_state=22122),
      min_features_to_select=7, n_jobs=-1, verbose=2)

In [131]:
selector.ranking_

array([1, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [132]:
X.columns

Index(['Butter', 'Cheese', 'Domestic milk intake', 'Fat content (Percent)',
       'Heifers 200-249kg', 'Heifers 250-299kg', 'Heifers 300-349kg',
       'Heifers 350-399kg', 'Heifers 400-449kg', 'Imported milk intake',
       'Skimmed milk sales', 'Skimmed milk powder', 'Whole milk sales',
       'Month'],
      dtype='object')

##### Define model with new data

In [133]:
rankings = selector.ranking_
columns_to_choose = [X.columns[i] for i in range(len(X.columns)) if rankings[i] == 1]
rfe_rfX_best = X[columns_to_choose]
rfe_rfX_best_train = X_train[columns_to_choose]
rfe_rfX_best_test = X_test[columns_to_choose]

##### Cross validation (both accuracy and absolute error)

In [134]:
(np.mean(cross_val_score(rf_best2, rfe_rfX, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf_best2, rfe_rfX, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   1.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s


[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   0.8s
[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s
[CV] END .................................................... total time=   0.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.2s finished


(0.10800966741991021, -3.6973292186639584)

###### Fit

In [135]:
rf_best2.fit(rfe_rfX_best_train, y_train)
ybrf2 = rf_best2.predict(rfe_rfX_best_test)

###### Results

In [136]:
ybrf2

array([34.92009036, 33.16597316, 30.99350879, 34.28137997, 32.35937164,
       34.81754985, 32.91581155, 39.43447693, 37.77316038, 37.22412135,
       28.41582694, 32.80939969, 37.04685965, 36.33094806, 35.59947944,
       32.74623028, 33.80669607, 35.4792382 , 31.09751731, 30.33332316,
       34.77613071, 29.86989799, 39.88952744, 34.03707153, 29.94740826,
       32.73158831, 35.2253438 , 37.59848428, 33.8285871 , 32.58901476,
       28.23487   , 30.13101173, 35.16411543, 33.46544315, 33.48256041,
       31.32242613, 28.95165853, 33.56234853, 29.52818973, 29.38699415,
       35.2661851 , 31.4480653 , 36.44696872, 33.09632399, 36.77805061])

###### Absolute error

In [137]:
mean_absolute_error(ybrf2, y_test)

3.1272075790569773

### K-Nearest Neighbour Regression

### 1. Basic test models with all features

In [138]:
#KNeighbors Regressor, default parameters
knn1 = KNeighborsRegressor(n_jobs=-1)

#KNeighbors Regressor, estimates from 10 nearest neighbours, each leaf has 10
#samples
knn2 = KNeighborsRegressor(n_neighbors=10, leaf_size=50, n_jobs=-1)

#KNeighbors Regressor, estimates from 10 nearest neighbours
knn3 = KNeighborsRegressor(n_neighbors=20, n_jobs=-1)

#### Cross validation tests (both accuracy and average error)

In [139]:
(np.mean(cross_val_score(knn1, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn1, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   2.9s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


(-0.041486826287366795, -3.891136483149816)

In [140]:
(np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


(-0.055084719315495946, -3.8821201601601594)

In [141]:
(np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


(-0.055084719315495946, -3.8821201601601594)

#### KNeighbors Regressor 1

###### Fit

In [142]:
knn1.fit(X_train, y_train)
yk1 = knn1.predict(X_test)

###### Results

In [143]:
yk1

array([30.828     , 32.574     , 29.66      , 34.882     , 33.816     ,
       32.496     , 36.614     , 37.486     , 41.954     , 38.596     ,
       29.72493814, 32.452     , 35.698     , 36.766     , 35.62      ,
       31.428     , 28.998     , 36.166     , 29.074     , 27.368     ,
       35.874     , 28.236     , 37.196     , 34.436     , 28.998     ,
       31.466     , 34.96      , 37.74      , 31.506     , 34.222     ,
       27.966     , 29.06      , 33.196     , 33.584     , 32.088     ,
       29.88893814, 30.892     , 31.642     , 28.874     , 28.61293814,
       35.776     , 28.94      , 35.008     , 33.486     , 39.624     ])

###### Absolute error

In [144]:
mean_absolute_error(yk1, y_test)

3.547931958625292

#### KNeighbors Regressor 2

###### Fit

In [145]:
knn2.fit(X_train, y_train)
yk2 = knn2.predict(X_test)

###### Results

In [146]:
yk2

array([33.52      , 32.913     , 29.309     , 35.145     , 32.146     ,
       32.661     , 33.836     , 38.235     , 39.575     , 38.624     ,
       32.24146907, 30.12046907, 37.37      , 35.913     , 35.874     ,
       33.4       , 31.562     , 35.535     , 32.164     , 28.533     ,
       36.739     , 29.892     , 39.002     , 33.631     , 30.989     ,
       32.65      , 38.117     , 37.873     , 32.661     , 32.021     ,
       29.993     , 29.892     , 31.085     , 33.885     , 33.175     ,
       32.24146907, 31.56      , 32.709     , 30.139     , 32.24146907,
       37.398     , 31.533     , 35.242     , 31.893     , 38.905     ])

###### Absolute error

In [147]:
mean_absolute_error(yk2, y_test)

3.555931958625292

#### KNeighbors Regressor 3

###### Fit

In [148]:
knn3.fit(X_train, y_train)
yk3 = knn3.predict(X_test)

###### Results

In [149]:
yk3

array([35.1745    , 32.5735    , 32.1555    , 37.001     , 32.209     ,
       32.204     , 35.9905    , 37.218     , 38.555     , 38.967     ,
       31.92223453, 30.94723453, 37.6235    , 36.84      , 34.6605    ,
       33.4035    , 32.8395    , 36.166     , 30.3535    , 30.281     ,
       36.286     , 32.04      , 37.6905    , 35.37      , 33.199     ,
       32.9895    , 37.778     , 37.369     , 32.6115    , 31.004     ,
       30.2615    , 32.0985    , 32.72273453, 34.653     , 32.568     ,
       31.29373453, 31.676     , 32.3595    , 32.106     , 31.92223453,
       36.1605    , 32.2905    , 34.616     , 32.209     , 37.686     ])

###### Absolute error

In [150]:
mean_absolute_error(yk3, y_test)

3.6299274341007672

### GridSearchCV

##### Parameter grid

In [151]:
# Loss
weights = ["uniform", "distance"]
# Number of estimators
n_neighbours = [1, 2, 5, 7, 10, 14, 15, 20]
# Criterion for measuring quality of split
p = [1, 2]
# Minimum number of samples required to split a node
leaf_size = [i for i in range(10, 50, 10)]
# Create the random grid
knn_grid = {'weights': weights,
            'n_neighbors': n_neighbours,
            'p': p,
            'leaf_size': leaf_size}

##### GridSearchCV algorithm

In [152]:
knn = KNeighborsRegressor(n_jobs=-1)

knn_gs = GridSearchCV(estimator = knn, param_grid = knn_grid,
                      scoring="neg_mean_absolute_error",
                      n_jobs=-1, cv=KFold(5), return_train_score=True,
                      verbose=4)

knn_gs.fit(X, y)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=KNeighborsRegressor(n_jobs=-1), n_jobs=-1,
             param_grid={'leaf_size': [10, 20, 30, 40],
                         'n_neighbors': [1, 2, 5, 7, 10, 14, 15, 20],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=4)

In [153]:
knn_gs.best_estimator_.get_params()

{'algorithm': 'auto',
 'leaf_size': 10,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': -1,
 'n_neighbors': 7,
 'p': 2,
 'weights': 'distance'}

### 2. Best model

##### Defining best model

In [154]:
knn_best = KNeighborsRegressor(algorithm='auto', leaf_size=10, 
                               metric='minkowski', metric_params=None,
                               n_neighbors=7,
                               n_jobs=-1, p=2, weights='distance')

##### Cross validation (both accuracy and absolute error)

In [155]:
(np.mean(cross_val_score(knn_best, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn_best, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s finished


(-0.01970503462869877, -3.805300153267807)

###### Fit

In [156]:
knn_best.fit(X_train, y_train)
ybknn = knn_best.predict(X_test)

###### Results

In [157]:
ybknn

array([31.65934829, 32.87490208, 29.60272644, 35.29974423, 33.94058158,
       33.0526848 , 36.37894423, 38.23067868, 40.60445723, 37.53704755,
       29.25724848, 32.25709335, 35.88198158, 35.57830727, 34.98432589,
       31.85086726, 30.38333202, 35.86249523, 29.69012748, 28.00150497,
       35.86786174, 28.28677424, 39.08564048, 34.37810827, 29.03564667,
       30.30262399, 37.4174427 , 38.56504195, 31.92717699, 31.59247317,
       29.19816196, 30.18606041, 31.62496828, 33.94736749, 34.09896795,
       29.04446885, 31.52848789, 32.23553546, 30.42016784, 29.29588386,
       37.37518797, 28.67399304, 35.19885864, 33.32337338, 39.09799776])

###### Absolute error

In [158]:
mean_absolute_error(ybknn, y_test)

3.5580441598370163

# Tests

### Test 1 (on df1)

In [24]:
df2

,Year,Month,Raw milk price,Butter,Calf nuts value,Cheese,Dairy meal value,Dairy nuts value,Domestic milk intake,Fat content (Percent),Imported milk intake,Maize meal value,Skimmed milk sales,Skimmed milk powder,Whole milk sales,Milk production volume
0,2007,1,28.30,3.8,0.0,1.3,0.0,0.0,123.0,3.87,54.6,0.0,12.3,2.4,32.1,177.6
1,2007,2,27.18,5.0,0.0,2.4,0.0,0.0,185.4,3.87,38.0,0.0,12.6,1.9,29.3,223.4
2,2007,3,25.64,10.1,0.0,10.2,0.0,0.0,386.5,3.80,35.8,0.0,12.8,5.0,32.4,422.3
3,2007,4,27.29,14.8,0.0,17.6,0.0,0.0,581.0,3.59,38.7,0.0,12.1,10.1,30.4,619.7
4,2007,5,29.76,19.1,0.0,19.3,0.0,0.0,688.3,3.62,46.9,0.0,12.8,13.2,31.8,735.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,2021,8,39.23,28.8,355.0,29.3,316.0,324.0,917.4,4.19,0.0,293.0,15.8,16.4,29.0,917.4
176,2021,9,42.44,26.5,360.0,33.2,323.0,332.0,776.7,4.43,0.0,299.0,15.1,11.1,26.6,776.7
177,2021,10,46.52,21.6,365.0,27.5,328.0,339.0,652.8,4.77,0.0,309.0,15.8,5.5,26.1,652.8
178,2021,11,48.65,17.8,370.0,20.9,333.0,344.0,460.6,4.90,0.0,314.0,15.4,0.0,25.5,460.6


In [39]:
X2 = df2[["Month"]+(list(df1.columns[3:-1]))]
y2 = df2["Raw milk price"]

stratify2 = df2["Month"]

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=22122,
                                                        stratify=stratify2)

In [40]:
X2

,Month,Butter,Calf nuts value,Cheese,Dairy meal value,Dairy nuts value,Domestic milk intake,Fat content (Percent),Imported milk intake,Maize meal value,Skimmed milk sales,Skimmed milk powder,Whole milk sales
0,1,3.8,0.0,1.3,0.0,0.0,123.0,3.87,54.6,0.0,12.3,2.4,32.1
1,2,5.0,0.0,2.4,0.0,0.0,185.4,3.87,38.0,0.0,12.6,1.9,29.3
2,3,10.1,0.0,10.2,0.0,0.0,386.5,3.80,35.8,0.0,12.8,5.0,32.4
3,4,14.8,0.0,17.6,0.0,0.0,581.0,3.59,38.7,0.0,12.1,10.1,30.4
4,5,19.1,0.0,19.3,0.0,0.0,688.3,3.62,46.9,0.0,12.8,13.2,31.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,8,28.8,355.0,29.3,316.0,324.0,917.4,4.19,0.0,293.0,15.8,16.4,29.0
176,9,26.5,360.0,33.2,323.0,332.0,776.7,4.43,0.0,299.0,15.1,11.1,26.6
177,10,21.6,365.0,27.5,328.0,339.0,652.8,4.77,0.0,309.0,15.8,5.5,26.1
178,11,17.8,370.0,20.9,333.0,344.0,460.6,4.90,0.0,314.0,15.4,0.0,25.5


In [41]:
X2_train

,Month,Butter,Calf nuts value,Cheese,Dairy meal value,Dairy nuts value,Domestic milk intake,Fat content (Percent),Imported milk intake,Maize meal value,Skimmed milk sales,Skimmed milk powder,Whole milk sales
116,9,18.5,295.0,21.9,251.0,271.0,583.8,4.33,56.7,230.0,17.2,7.7,27.2
38,3,10.1,0.0,15.5,0.0,0.0,382.1,3.89,33.9,0.0,15.4,1.3,29.1
155,12,10.9,312.0,5.4,266.0,286.0,241.4,4.58,0.0,230.0,16.1,10.2,25.8
160,5,29.8,320.0,31.6,273.0,291.0,1115.3,3.87,0.0,232.0,15.7,24.6,31.6
93,10,13.5,279.0,14.8,250.0,270.0,426.3,4.49,37.4,227.0,16.5,0.0,23.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,8,20.3,295.0,23.3,252.0,270.0,699.3,4.08,50.8,232.0,18.2,11.0,26.8
48,1,4.0,0.0,2.5,0.0,0.0,146.7,3.99,28.8,0.0,14.5,0.0,27.6
108,1,5.0,296.0,2.0,264.0,272.0,147.6,4.17,78.6,219.0,17.8,4.4,24.7
154,11,15.1,312.0,23.5,267.0,286.0,418.1,4.84,0.0,229.0,16.9,0.0,26.8


In [42]:
y2_test

14     39.64
148    31.47
62     33.41
133    37.00
173    37.39
175    39.23
74     34.09
131    40.59
69     35.74
57     38.26
1      27.18
34     28.63
92     35.45
71     33.70
87     38.55
150    31.08
104    29.23
84     42.34
9      44.60
63     30.88
144    34.28
42     31.20
166    39.14
52     32.05
122    31.66
127    37.68
145    34.38
70     35.93
137    31.66
44     33.67
30     22.04
5      32.64
46     34.18
151    31.95
163    33.89
11     43.25
27     22.34
136    31.17
4      29.76
0      28.30
85     41.76
111    24.08
141    38.75
174    37.20
68     33.31
Name: Raw milk price, dtype: float64

#### Random Forest

##### 1. Basic test to test for suitability for remaining sections

In [43]:
#Random forest(basic features, randomized state, squared error scoring)
rf21 = RandomForestRegressor(criterion="absolute_error")

#Random forest, 500 estimators, 20 samples to split a node, min 5 samples at
#leaf nodes, squared error scoring, non-random state set
rf22 = RandomForestRegressor(n_estimators=500, min_samples_split=20, 
                             min_samples_leaf=5, n_jobs=-1, 
                             criterion="absolute_error", random_state=22122)

#Random forest, 1000 estimators, squared error scoring, non-random state set
rf23 = RandomForestRegressor(n_estimators=1000, n_jobs=-1,
                            criterion="absolute_error", random_state=22122)

##### Cross-validation tests (accuracy and mean average error)

In [46]:
(np.mean(cross_val_score(rf21, X2, y2, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf21, X2, y2, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   1.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   1.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.7s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   1.0s
[CV] END .................................................... total time=   0.9s
[CV] END .................................................... total time=   0.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.2s finished


(-0.02740591114193087, -4.008099879546213)

In [47]:
(np.mean(cross_val_score(rf22, X2, y2, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf22, X2, y2, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   2.3s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV] END .................................................... total time=   2.3s
[CV] END .................................................... total time=   2.3s
[CV] END .................................................... total time=   2.2s
[CV] END .................................................... total time=   2.3s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   11.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   2.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s


[CV] END .................................................... total time=   2.2s
[CV] END .................................................... total time=   2.1s
[CV] END .................................................... total time=   1.5s
[CV] END .................................................... total time=   1.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    9.5s finished


(0.12053912203964903, -3.6897142114781403)

In [48]:
(np.mean(cross_val_score(rf23, X2, y2, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf23, X2, y2, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   4.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.2s remaining:    0.0s


[CV] END .................................................... total time=   4.2s
[CV] END .................................................... total time=   4.3s
[CV] END .................................................... total time=   4.3s
[CV] END .................................................... total time=   4.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   21.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   4.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.1s remaining:    0.0s


[CV] END .................................................... total time=   4.3s
[CV] END .................................................... total time=   4.3s
[CV] END .................................................... total time=   4.3s
[CV] END .................................................... total time=   4.3s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   21.3s finished


(0.007808658041367722, -3.9039427363863495)

In [52]:
rf21.fit(X2_train, y2_train)
y21 = rf21.predict(X2_test)

In [53]:
y21

array([30.75534072, 31.8243    , 32.52289691, 34.3424    , 35.8764    ,
       37.1562    , 32.38785   , 39.6336    , 39.77564691, 38.7936    ,
       34.07064691, 36.922     , 35.9444    , 32.97884072, 34.4404    ,
       31.5293    , 32.3065    , 36.0591    , 35.75618763, 34.0924    ,
       33.8683    , 32.7704    , 37.8796    , 33.7226    , 31.7246    ,
       32.3197    , 34.9943    , 38.87      , 31.1943    , 34.3137    ,
       28.2884    , 30.3271    , 39.45444691, 32.68775   , 32.73285   ,
       35.1264567 , 28.4047    , 31.0911    , 29.9584    , 33.64709381,
       36.276     , 27.74155   , 36.6248    , 36.3817    , 36.7413    ])

In [57]:
mean_absolute_error(y2_test, y21)

2.9341535969302623

In [60]:
rf21.score(X2_test, y2_test)

0.3989811905507652

In [ ]:
rf2.fit(X_train, y_train)
y2 = rf2.predict(X_test)

In [ ]:
y2

array([33.09226, 32.61903, 34.30065, 34.29917, 37.25723, 36.69729,
       31.83883, 35.47793, 31.7087 , 33.44176, 34.88732, 32.31794,
       35.42382, 32.24653, 31.8682 , 32.66066, 37.77001, 34.95742,
       35.20645, 35.7047 , 37.72055, 31.72545, 32.93658, 33.64466])

In [ ]:
mean_absolute_error(y_test, y2)

2.9999199999999733

In [ ]:
rf3.fit(X_train, y_train)
y3 = rf3.predict(X_test)

In [ ]:
y3

array([32.71772 , 32.21856 , 35.143655, 34.973405, 37.96083 , 36.71873 ,
       31.11904 , 34.94118 , 31.21576 , 33.60151 , 34.65927 , 31.330905,
       34.53934 , 30.48466 , 30.94582 , 31.2412  , 38.21911 , 33.91221 ,
       35.824785, 36.482805, 41.60696 , 29.47778 , 34.03515 , 37.134435])

In [ ]:
mean_absolute_error(y_test, y3)

2.1407187500000098

In [ ]:
rf4 = clone(rf3)

In [ ]:
#Create Random Feature elimination for above model
selector1 = RFECV(rf4, cv=5, step=1, n_jobs=-1, verbose=2, min_features_to_select=7)

#Fit all data
selector1.fit(X, y)

Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.


RFECV(cv=5,
      estimator=RandomForestRegressor(criterion='absolute_error',
                                      n_estimators=1000, n_jobs=-1,
                                      random_state=22122),
      min_features_to_select=7, n_jobs=-1, verbose=2)

In [ ]:
selector1.ranking_

array([2, 1, 6, 1, 4, 5, 1, 1, 1, 3, 1, 1, 7])

In [ ]:
X.columns

Index(['Butter', 'Calf nuts value', 'Cheese', 'Dairy meal value',
       'Dairy nuts value', 'Domestic milk intake', 'Fat content (Percent)',
       'Imported milk intake', 'Maize meal value', 'Skimmed milk sales',
       'Skimmed milk powder', 'Whole milk sales', 'Month'],
      dtype='object')

In [ ]:
rankings = selector1.ranking_
columns_to_choose = [X.columns[i] for i in range(len(X.columns)) if rankings[i] == 1]
rfe_rfX = X[columns_to_choose]
rfe_rfX_train = X_train[columns_to_choose]
rfe_rfX_test = X_test[columns_to_choose]

In [ ]:
columns_to_choose

['Calf nuts value',
 'Dairy meal value',
 'Fat content (Percent)',
 'Imported milk intake',
 'Maize meal value',
 'Skimmed milk powder',
 'Whole milk sales']

In [ ]:
(np.mean(cross_val_score(rf4, rfe_rfX, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf4, rfe_rfX, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.4s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.7s
[CV] END .................................................... total time=   0.4s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time=   0.6s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] END .................................................... total time=   0.6s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s
[CV] END .................................................... total time=   0.4s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.7s finished


(-0.549049869391234, -4.076352097368378)

In [ ]:
rf4.fit(rfe_rfX_train, y_train)
y4 = rf4.predict(rfe_rfX_test)

In [ ]:
y4

array([32.60927 , 31.18014 , 35.145035, 35.31445 , 40.13027 , 36.878245,
       31.97339 , 35.773065, 31.13415 , 33.11691 , 34.86476 , 30.73545 ,
       34.28409 , 31.29424 , 30.42335 , 30.27531 , 37.989285, 33.75604 ,
       36.09614 , 37.02593 , 41.199515, 29.25163 , 34.35284 , 37.46068 ])

In [ ]:
mean_absolute_error(y4, y_test)

1.9280364583333707

In [ ]:
n_estimators = [i for i in range(100, 1001, 100)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [i for i in range(10, 101, 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3, 4, 5, 7, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Criterion to select scoring
criterion = ["absolute_error", "squared_error", "gini"]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion}

In [ ]:
rf = RandomForestRegressor(random_state=22122,
                           n_jobs=-1)

gs_random = GridSearchCV(estimator = rf, param_grid = random_grid,
                               scoring="neg_mean_absolute_error",
                               n_jobs=-1, cv=KFold(5), return_train_score=True,
                               verbose=4)

gs_random.fit(X, y)

Fitting 5 folds for each of 23760 candidates, totalling 118800 fits


KeyboardInterrupt: 

In [ ]:
rf_best = clone(gs_random.best_estimator_)

##### Cross validation (both accuracy and absolute error)

In [ ]:
(np.mean(cross_val_score(rf_best, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf_best, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

###### Fit

In [ ]:
rf_best.fit(X_train, y_train)
ybrf = rf4.predict(X_test)

###### Results

In [ ]:
ybrf

###### Absolute error

In [ ]:
mean_absolute_error(ybrf, y_test)

##### 4. Best model from 3 with RFE applied

##### Apply RFE (with cross-validation) 

In [ ]:
#Clone best performing model of previous section
rf_best2 = clone(rf_best)

#Create Random Feature elimination for above model
selector = RFECV(rf_best2, cv=5, step=1, n_jobs=-1, verbose=2, min_features_to_select=len(X.columns) // 2)

#Fit all data
selector.fit(X, y)

In [ ]:
selector.ranking_

In [ ]:
X.columns

##### Define model with new data

In [ ]:
rankings = selector.ranking_
columns_to_choose = [X.columns[i] for i in range(len(X.columns)) if rankings[i] == 1]
rfe_rfX_best = X[columns_to_choose]
rfe_rfX_best_train = X_train[columns_to_choose]
rfe_rfX_best_test = X_test[columns_to_choose]

##### Cross validation (both accuracy and absolute error)

In [ ]:
(np.mean(cross_val_score(rf_best2, rfe_rfX, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(rf_best2, rfe_rfX, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

###### Fit

In [ ]:
rf_best2.fit(rfe_rfX_best_train, y_train)
ybrf2 = rf4.predict(rfe_rfX_best_test)

###### Results

In [ ]:
ybrf2

###### Absolute error

In [ ]:
mean_absolute_error(ybrf2, y_test)

### K-Nearest Neighbour Regression

### 1. Basic test models with all features

In [ ]:
#KNeighbors Regressor, default parameters
knn1 = KNeighborsRegressor(n_jobs=-1)

#KNeighbors Regressor, estimates from 10 nearest neighbours, each leaf has 10
#samples
knn2 = KNeighborsRegressor(n_neighbors=10, leaf_size=50, n_jobs=-1)

#KNeighbors Regressor, estimates from 10 nearest neighbours
knn3 = KNeighborsRegressor(n_neighbors=20, n_jobs=-1)

#### Cross validation tests (both accuracy and average error)

In [ ]:
(np.mean(cross_val_score(knn1, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn1, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.7s finished


(0.9180921608506607, -368.379723482743)

In [ ]:
(np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished


(0.9247140939410677, -350.91984476224167)

In [ ]:
(np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn2, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s
[CV] END .................................................... total time=   0.1s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished


(0.9247140939410677, -350.91984476224167)

#### KNeighbors Regressor 1

###### Fit

In [ ]:
knn1.fit(X_train, y_train)
yk1 = knn1.predict(X_test)

###### Results

In [ ]:
yk1

array([4944.2,  692.8, 2102.6, 3167.2, 3299.8, 5318. , 6853.8,  668.6,
       1491. , 4594.8, 7215. , 6197.6, 4340.8, 4506. , 5739.2, 1372.8,
       1601.6,  571. , 3981.2, 1243.4, 6257. , 4437.2,  686.2, 5202.8,
       2200. , 6881.8,  550.4, 5128.2, 5024.4, 4883.8, 6692. , 5513.4,
       2963.8, 6775.8, 5420.2, 6517.4, 4489. , 1334.8, 3688.4, 1923.4,
       4225.4, 5793.6, 2972.6, 1856.8, 5006.2, 3305.6,  933.4, 2379.2,
       6778.8, 6631.6, 5943. , 6861.8, 4980. , 4766.2, 6220.2, 5546.2,
       5636.2,  691. , 1980.6, 1650.2, 1595.2, 4388.4, 5740.6, 4904.6,
       6417.4, 5408.6, 3682.8,  601.4, 5088.4, 2728.8, 3186.2, 5343.8,
        493.2,  673.2, 3244. , 3715.4, 1555.8, 4296.6, 1588.2, 3071.8,
       6630.8, 3454.2, 1852.8, 6622.2, 1722. , 5907.2, 5768.8, 1785.6,
       2684.2, 6338.6, 4792. , 2179. , 2830.6, 4069.2, 5125.4, 6479.8,
       6954.4, 3680.8, 3090. , 4081.4, 2904.6, 2272.2, 1492. , 3108.8,
       6630.6, 6145.4, 6076.6, 6489. , 2804.6, 5221.6, 6508. , 4055.8,
      

###### Absolute error

In [ ]:
mean_absolute_error(yk1, y_test)

311.5599169262721

#### KNeighbors Regressor 2

###### Fit

In [ ]:
knn2.fit(X_train, y_train)
yk2 = knn2.predict(X_test)

###### Results

In [ ]:
yk2

array([4911.3,  690.3, 2109.4, 3204.6, 3353.4, 5688.3, 6481.2,  692.5,
       1533.9, 4616.5, 7034.2, 5783.6, 4442.9, 4831.4, 5801.3, 1341.8,
       1566.8,  576.4, 4167.7, 1325.5, 6277.8, 4377.6,  669.8, 5185.4,
       2063.1, 6811.3,  592.6, 5090.9, 5224.2, 5037.7, 6529.5, 5418.8,
       3002. , 6761.3, 5423.6, 6033.8, 4664.3, 1322.8, 3648.3, 1907. ,
       4281.4, 5391.1, 2823.7, 1798.8, 5044.9, 3241.6,  864.5, 2440.8,
       6385.3, 6340. , 6213.1, 6885.7, 5334.2, 4661.3, 5971.5, 5202. ,
       5381.2,  668.2, 1906.9, 1575.3, 1601.4, 4173.6, 5617.6, 4949.6,
       6412.9, 5451.4, 3615.6,  575.6, 5138.3, 2750.7, 3124. , 5390.1,
        436. ,  688.7, 3199.6, 3832. , 1567.3, 4376.6, 1530.8, 3396. ,
       6644.2, 3470.2, 1930.1, 6505.5, 1789.8, 5910.8, 5534.3, 1813.4,
       2694.5, 6161.2, 4858.6, 2307.6, 2791.4, 4184.7, 4945.7, 6347.7,
       6942.9, 3862.2, 2836.6, 3897.4, 2968.4, 2194.8, 1522.8, 3066.3,
       6534.1, 5708.3, 5353.6, 6218.7, 2856.6, 5272. , 5964.3, 4004.4,
      

###### Absolute error

In [ ]:
mean_absolute_error(yk2, y_test)

306.98068535825547

#### KNeighbors Regressor 3

###### Fit

In [ ]:
knn3.fit(X_train, y_train)
yk3 = knn3.predict(X_test)

###### Results

In [ ]:
yk3

array([5178.35,  671.2 , 2075.75, 3040.35, 3402.15, 5539.  , 6439.  ,
        695.  , 1582.65, 4600.4 , 6848.95, 5727.  , 4457.8 , 5088.  ,
       5802.15, 1346.95, 1609.3 ,  564.35, 3972.7 , 1377.5 , 6279.7 ,
       4416.5 ,  672.25, 4971.  , 2106.4 , 6635.3 ,  618.8 , 5276.05,
       5253.9 , 4903.1 , 6425.75, 5540.15, 3189.4 , 7008.85, 5453.65,
       6225.15, 4547.2 , 1299.75, 3713.2 , 1892.3 , 4417.6 , 5407.35,
       2832.45, 1828.4 , 5349.6 , 3239.25,  837.  , 2510.1 , 6577.15,
       6403.35, 6375.6 , 6727.4 , 5129.35, 4661.15, 5929.3 , 5308.25,
       5312.05,  650.15, 1946.55, 1590.2 , 1579.95, 4108.4 , 5538.15,
       5082.85, 6339.55, 5742.15, 3711.95,  581.65, 5494.15, 2754.1 ,
       3134.75, 5558.25,  426.95,  691.4 , 3416.95, 3764.4 , 1573.25,
       4191.2 , 1524.55, 4070.9 , 6717.9 , 3486.5 , 2012.3 , 6337.1 ,
       1777.05, 5811.55, 5448.25, 1946.1 , 2704.1 , 6256.5 , 4760.8 ,
       2287.4 , 2717.2 , 4048.3 , 4512.7 , 6378.55, 6944.5 , 3864.6 ,
       2861.1 , 3840

###### Absolute error

In [ ]:
mean_absolute_error(yk3, y_test)

301.3296469366563

### GridSearchCV

##### Parameter grid

In [ ]:
# Loss
weights = ["uniform", "distance"]
# Number of estimators
n_neighbours = [1, 2, 5, 7, 10, 14, 15, 20]
# Criterion for measuring quality of split
p = [1, 2]
# Minimum number of samples required to split a node
leaf_size = [i for i in range(10, 50, 10)]
# Create the random grid
knn_grid = {'weights': weights,
            'n_neighbors': n_neighbours,
            'p': p,
            'leaf_size': leaf_size}

##### GridSearchCV algorithm

In [ ]:
knn = KNeighborsRegressor(n_jobs=-1)

knn_gs = GridSearchCV(estimator = knn, param_grid = knn_grid,
                      scoring="neg_mean_absolute_error",
                      n_jobs=-1, cv=KFold(5), return_train_score=True,
                      verbose=4)

knn_gs.fit(X, y)

Fitting 5 folds for each of 128 candidates, totalling 640 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=KNeighborsRegressor(n_jobs=-1), n_jobs=-1,
             param_grid={'leaf_size': [10, 20, 30, 40],
                         'n_neighbors': [1, 2, 5, 7, 10, 14, 15, 20],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             return_train_score=True, scoring='neg_mean_absolute_error',
             verbose=4)

In [ ]:
knn_gs.best_estimator_.get_params()

{'algorithm': 'auto',
 'leaf_size': 10,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': -1,
 'n_neighbors': 20,
 'p': 1,
 'weights': 'distance'}

### 2. Best model

##### Defining best model

In [ ]:
knn_best = KNeighborsRegressor(algorithm='auto', leaf_size=10, 
                               metric='minkowski', metric_params=None, 
                               n_jobs=-1, p=1, weights='distance')

##### Cross validation (both accuracy and absolute error)

In [ ]:
(np.mean(cross_val_score(knn_best, X, y, cv=KFold(5), verbose=2)),
 np.mean(cross_val_score(knn_best, X, y, cv=KFold(5), verbose=2, scoring="neg_mean_absolute_error")))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s
[CV] END .................................................... total time=   0.2s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.9s finished


(0.9238901692717343, -346.3115188195462)

###### Fit

In [ ]:
knn_best.fit(X_train, y_train)
ybknn = knn_best.predict(X_test)

###### Results

In [ ]:
ybknn

array([5376.44979017,  701.87257425, 2153.51084861, 3161.75779251,
       3334.66238808, 5343.21560057, 6250.23689914,  675.00320328,
       1496.26840723, 4304.83403693, 7140.27682663, 6200.1926661 ,
       4648.13800993, 4554.63431667, 5539.92262326, 1399.84225884,
       1617.86121026,  566.03552037, 4162.23938629, 1222.5059959 ,
       6269.71984667, 4490.68095016,  677.91166079, 5336.6500824 ,
       2445.38607356, 6905.20173927,  340.86180087, 5109.01010211,
       5327.22182454, 5154.64671932, 6346.03993582, 5515.24103973,
       2917.13351733, 6677.33379093, 5613.70154255, 6525.10301698,
       4291.19712643, 1249.52236649, 3666.96027792, 1849.03001107,
       4271.46528147, 5655.68225431, 2615.0679525 , 1847.7436953 ,
       5142.28434374, 3187.04836719,  947.55259071, 2242.50985428,
       6777.34732875, 6197.96978902, 6440.72152239, 6774.8255985 ,
       5023.9617116 , 4913.9856626 , 6130.83415221, 5559.73018772,
       5446.7599344 ,  694.9110202 , 1975.98957356, 1654.19028

###### Absolute error

In [ ]:
mean_absolute_error(ybknn, y_test)

290.824070365135